In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage

# Configuration
load_dotenv()
llm = ChatMistralAI(model="mistral-small-latest", mistral_api_key=os.getenv("MISTRAL_API_KEY"))

def get_params(genre=None, tempo=None):
    """Trouve les paramètres optimaux"""
    df = pd.read_csv('dataset.csv')
    top_songs = df[df['popularity'] >= df['popularity'].quantile(0.8)]
    
    filtered = top_songs.copy()
    if genre:
        genre_songs = filtered[filtered['track_genre'] == genre]
        if len(genre_songs) > 10:
            filtered = genre_songs
    
    if tempo:
        # FILTRAGE PAR TEMPO PRÉCIS (±10 BPM)
        tempo_range = (tempo - 10, tempo + 10)
        tempo_songs = filtered[(filtered['tempo'] >= tempo_range[0]) & (filtered['tempo'] <= tempo_range[1])]
        if len(tempo_songs) > 5:
            filtered = tempo_songs
    
    if len(filtered) < 5:
        filtered = top_songs
    
    params = filtered[['danceability', 'energy', 'key', 'mode', 'acousticness', 
                       'instrumentalness', 'valence', 'tempo', 'duration_ms']].median()
    
    # FORCER LE TEMPO DEMANDÉ
    if tempo:
        params['tempo'] = tempo
    
    return params.to_dict(), len(filtered)

def ask_ai(prompt):
    """Pose une question à l'IA"""
    return llm.invoke([HumanMessage(content=prompt)]).content

def get_user_intent(user_input):
    """Détecte genre et tempo dans la demande"""
    # Essayer d'abord d'extraire le tempo numérique
    import re
    tempo_match = re.search(r'(\d+)\s*(?:bpm|BPM|tempo)', user_input)
    tempo = int(tempo_match.group(1)) if tempo_match else None
    
    # Demander à l'IA pour le genre
    prompt = f"Extrait juste le genre musical de: '{user_input}'. Réponds UNIQUEMENT avec le nom du genre ou 'tout'."
    genre = ask_ai(prompt).strip().lower()
    genre = None if genre == 'tout' else genre
    
    return genre, tempo

def format_params(params):
    """Formatage simple des paramètres"""
    key_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    return {
        "Tonalité": f"{key_names[int(params['key'])]} {'majeur' if params['mode'] > 0.5 else 'mineur'}",
        "Tempo": f"{params['tempo']:.0f} BPM",
        "Énergie": f"{params['energy']*100:.0f}%",
        "Danse": f"{params['danceability']*100:.0f}%",
        "Humeur": f"{params['valence']*100:.0f}%",
        "Acoustique": f"{params['acousticness']*100:.0f}%",
        "Durée": f"{params['duration_ms']/60000:.1f} min"
    }

# Interface principale
print("=== Générateur de Paramètres Musicaux ===")
user_input = input("\nQuelle musique veux-tu créer ? (ex: 'rock à 120 BPM', 'pop 90') ")

genre, tempo = get_user_intent(user_input)

print(f"\nAnalyse pour: ", end="")
if genre:
    print(f"genre '{genre}'", end=" ")
if tempo:
    print(f"tempo {tempo} BPM", end=" ")
if not genre and not tempo:
    print("tout style", end=" ")

params, sample_size = get_params(genre, tempo)

formatted = format_params(params)

print(f"\n\nParamètres optimaux ({sample_size} morceaux similaires):")
for k, v in formatted.items():
    print(f"  • {k}: {v}")

# Génération des conseils
prompt = f"""
Donne 3 conseils concis pour composer une musique:
- Genre: {genre if genre else 'populaire'}
- Tempo: {params['tempo']:.0f} BPM
- Tonalité: {formatted['Tonalité']}
- Énergie: {params['energy']:.2f}/1.0
- Humeur: {params['valence']:.2f}/1.0

Conseils demandés:
1. Rythme et structure adaptée à {params['tempo']:.0f} BPM
2. Accords et mélodie en {formatted['Tonalité']}
3. Production et ambiance
"""

advice = ask_ai(prompt)
print(f"\nConseils pour {params['tempo']:.0f} BPM:\n{advice}")

=== Générateur de Paramètres Musicaux ===

Analyse pour: genre 'pop' tempo 100 BPM 

Paramètres optimaux (201 morceaux similaires):
  • Tonalité: F mineur
  • Tempo: 100 BPM
  • Énergie: 63%
  • Danse: 70%
  • Humeur: 49%
  • Acoustique: 30%
  • Durée: 3.4 min

Conseils pour 100 BPM:
1. **Rythme et structure** :
   - Utilise une grille rythmique pop dynamique mais modérée (énergie 0.63) avec des patterns de batterie simples (kick sur les temps, snare en backbeat, hi-hats en 8èmes ou 16èmes).
   - Structure classique (couplet/pré-refrain/refrain) avec des transitions fluides et un refrain mémorable.

2. **Accords et mélodie** :
   - Base tonale : Fm (tonique), D♭ (sous-dominante), Cm (dominante) pour une ambiance mélancolique. Ajoute des accords comme B♭m (VI) ou E♭m (III) pour de la couleur.
   - Mélodie : Phrases courtes et répétitives, avec des sauts de tierce ou quarte pour une touche pop, et des notes tenues pour renforcer l’émotion (humeur 0.49).

3. **Production et ambiance** :
 